In [1]:
from census_data_collector import get_county_info
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot  as plt
%matplotlib inline


In [2]:
path = os.getcwd() 
data_root = path + '\\DATA\\'
data_root

'C:\\Users\\Kaipable\\Desktop\\GitHub\\voter-protection-corps\\pollworkers_src\\DATA\\'

In [3]:
my_api_key = 'fill in with your API key"

In [4]:
# Define Function Inputs
census_vars = ['DP05_0033E','DP05_0038E','DP05_0072E','DP03_0063E','DP02_0001E','DP03_0062E','DP05_0018E','DP05_0014E','DP05_0015E','DP05_0016E','DP05_0017E']

str_vars = ",".join([str(v) for v in census_vars])

state_code = '*'

In [5]:
#headers, data = cdc.get_county_info(state_code, str_vars, my_api_key)

In [6]:
import censusdata

In [7]:
census = censusdata.download('acs5', 2018, censusdata.censusgeo([('county', '*')]),
                                  census_vars,tabletype='profile')

In [8]:
print(len(census))
census.reset_index(level=0, inplace=True)
census.head()

3220


,index,DP05_0033E,DP05_0038E,DP05_0072E,DP03_0063E,DP02_0001E,DP03_0062E,DP05_0018E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E
0,"Washington County, Mississippi: Summary level:...",47086,33982,579,49433,18299.0,30834,36.9,3291,4282,1961,710
1,"Perry County, Mississippi: Summary level: 050,...",12028,2368,150,55077,4563.0,39007,40.9,897,1247,589,236
2,"Choctaw County, Mississippi: Summary level: 05...",8321,2596,14,49162,3164.0,37203,44.0,644,979,547,214
3,"Itawamba County, Mississippi: Summary level: 0...",23480,1681,320,53717,8706.0,40510,40.0,1435,2250,1181,574
4,"Carroll County, Mississippi: Summary level: 05...",10129,3503,2,57919,3658.0,43060,47.0,623,1353,726,170


In [9]:
census.iloc[1,0].params()[0][1] +census.iloc[1,0].params()[1][1]

'28111'

In [10]:
# FIPS and Join Info
census['fips_code'] = census['index'].apply(lambda x: x.params()[0][1] + x.params()[1][1])
census['state_code'] = census['index'].apply(lambda x: x.params()[0][1])

census.head()

,index,DP05_0033E,DP05_0038E,DP05_0072E,DP03_0063E,DP02_0001E,DP03_0062E,DP05_0018E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,fips_code,state_code
0,"Washington County, Mississippi: Summary level:...",47086,33982,579,49433,18299.0,30834,36.9,3291,4282,1961,710,28151,28
1,"Perry County, Mississippi: Summary level: 050,...",12028,2368,150,55077,4563.0,39007,40.9,897,1247,589,236,28111,28
2,"Choctaw County, Mississippi: Summary level: 05...",8321,2596,14,49162,3164.0,37203,44.0,644,979,547,214,28019,28
3,"Itawamba County, Mississippi: Summary level: 0...",23480,1681,320,53717,8706.0,40510,40.0,1435,2250,1181,574,28057,28
4,"Carroll County, Mississippi: Summary level: 05...",10129,3503,2,57919,3658.0,43060,47.0,623,1353,726,170,28015,28


In [11]:
# Get proper mapping 
state_code_map = pd.read_csv(data_root +'state_code_map.csv', dtype=str)
state_code_map.head()

,Name,FIPS State Numeric Code,Official USPS Code
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA


In [12]:
full_census = census.merge(state_code_map, how="left", right_on = "FIPS State Numeric Code", left_on='state_code')
full_census.head()

,index,DP05_0033E,DP05_0038E,DP05_0072E,DP03_0063E,DP02_0001E,DP03_0062E,DP05_0018E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,fips_code,state_code,Name,FIPS State Numeric Code,Official USPS Code
0,"Washington County, Mississippi: Summary level:...",47086,33982,579,49433,18299.0,30834,36.9,3291,4282,1961,710,28151,28,Mississippi,28,MS
1,"Perry County, Mississippi: Summary level: 050,...",12028,2368,150,55077,4563.0,39007,40.9,897,1247,589,236,28111,28,Mississippi,28,MS
2,"Choctaw County, Mississippi: Summary level: 05...",8321,2596,14,49162,3164.0,37203,44.0,644,979,547,214,28019,28,Mississippi,28,MS
3,"Itawamba County, Mississippi: Summary level: 0...",23480,1681,320,53717,8706.0,40510,40.0,1435,2250,1181,574,28057,28,Mississippi,28,MS
4,"Carroll County, Mississippi: Summary level: 05...",10129,3503,2,57919,3658.0,43060,47.0,623,1353,726,170,28015,28,Mississippi,28,MS


In [13]:
len(full_census)

3220

In [18]:
# Check for Single State
wi_df = full_census[full_census['Name']=='Wisconsin']
print(len(wi_df))

wi_df.head()

72


,index,DP05_0033E,DP05_0038E,DP05_0072E,DP03_0063E,DP02_0001E,DP03_0062E,DP05_0018E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,fips_code,state_code,Name,FIPS State Numeric Code,Official USPS Code
822,"Rock County, Wisconsin: Summary level: 050, st...",161769,7081,11554,69229,64538.0,55832,39.5,10369,14573,7672,3283,55105,55,Wisconsin,55,WI
823,"Waushara County, Wisconsin: Summary level: 050...",24116,435,1348,65597,9869.0,51195,49.0,2061,3267,1705,652,55137,55,Wisconsin,55,WI
824,"Adams County, Wisconsin: Summary level: 050, s...",20073,568,538,57609,8619.0,43280,53.3,2009,3420,1653,546,55001,55,Wisconsin,55,WI
825,"Bayfield County, Wisconsin: Summary level: 050...",14992,125,193,65004,6963.0,52910,52.0,1622,2461,1104,326,55007,55,Wisconsin,55,WI
826,"Lafayette County, Wisconsin: Summary level: 05...",16735,107,572,70488,6679.0,59629,41.4,1247,1531,928,482,55065,55,Wisconsin,55,WI
827,"Monroe County, Wisconsin: Summary level: 050, ...",45502,762,1309,69539,17772.0,58178,39.5,3142,4264,2072,1001,55081,55,Wisconsin,55,WI
828,"Pierce County, Wisconsin: Summary level: 050, ...",41603,188,454,85999,15367.0,72111,36.8,2861,3426,1538,728,55093,55,Wisconsin,55,WI
829,"Wood County, Wisconsin: Summary level: 050, st...",73274,444,1580,68235,32245.0,53473,43.8,4717,7786,4328,2322,55141,55,Wisconsin,55,WI
830,"Ashland County, Wisconsin: Summary level: 050,...",15712,110,213,55692,6546.0,43861,41.8,1183,1663,894,307,55003,55,Wisconsin,55,WI
831,"Calumet County, Wisconsin: Summary level: 050,...",49807,420,1782,86147,19567.0,72530,40.1,3188,4063,2030,840,55015,55,Wisconsin,55,WI


In [28]:
# Write CSV
full_census.to_csv('all_states_census.csv')